In [1]:
import sys
sys.path.append('/data/common')
from lib import *
from statsmodels.stats.proportion import proportion_effectsize
from statsmodels.stats.power import zt_ind_solve_power
from scipy import stats as st
from statsmodels.stats.proportion import proportions_ztest
from statsmodels.stats.proportion import proportion_confint

## Defining the Duration of an Experiment Based on the Desired Minimum Detectable Effect (MDE)

### Gathering data on cr from open_phones to calls

In [2]:
def get_openphones_and_calls(city, month):
    if city is None:
        print('CR from openphones to calls, All cities')
        city_clickhouse = ''
        city_vertica = ''
    else:
        print('CR from openphones to calls, ' + city)
        city_clickhouse = f"and seotarget_city in ('{city}')"
        city_vertica = f"and city in ('{city}')"
    
    openphones = clickhouse(f"""
        select *
        from
        (
        select session_id, event_time catalog_time
        from zoon.stat
        where date_trunc('month', event_date) = '{month}'
            and page_url like '%zoon.%'
            and seotarget_type = 'catalog'
            and ev_type = 'pageview'
            and user_class = '' -- добавил позже
            and ua_type = 'desktop' -- добавил позже
            {city_clickhouse}
            ) as catalogs
        inner join
        (select session_id, event_time open_phone_time, event_date, anon_user_id, object_id as org_id
        from zoon.stat
        where date_trunc('month',event_date) = '{month}'
            and page_url like '%zoon.%'
            and ev_type = 'event'
            {city_clickhouse}
            and ua_type = 'desktop'
            and ev_action = 'open'
            and ev_category like 'phone_o%'
            and ev_label not like '%standard%'
            and session_id in (
                select distinct session_id
                from zoon.stat
                where date_trunc('month',event_date) = '{month}'
                    and page_url like '%zoon.%'
                    and ev_type = 'pageview'
                    {city_clickhouse}
            and ua_type = 'desktop'
            and user_class = ''  -- добавил позже
            and seotarget_type = 'catalog') as catalogs_2
             ) as openphones
        using(session_id)
        where catalog_time < open_phone_time
""")
    

    
    calls = vertica(f"""
    with cits as (
        select distinct _id 
        from zoon.organization 
        where 1=1
            {city_vertica}
    )

    select  
        _id, 
        ts, 
        date_trunc('day', ts) as event_date,
        duration,
        profit_type,
        "approx_user.anon_user_id" as anon_user_id,
        org_id
    from zoon.stat_mangotelecom
    where date_trunc('month', ts) = '{month}'
        and org_id in (select * from cits)
        and "real_type.object_id" is null 
        -- and source is null
        and org_phone != phone_from 
        and "approx_user.anon_user_id" is not null
        and "approx_user.anon_user_id" != ''
    """)
    
    openphones = openphones.drop_duplicates(['session_id', 'open_phone_time', 'anon_user_id', 'org_id'])
    df = openphones.merge(calls, how='inner', on=['anon_user_id', 'org_id', 'event_date']).drop_duplicates()
    calls_after_openphone = df['_id'].nunique()
    openphones_not_standart = len(openphones)
    cv_from_openphones_to_calls = calls_after_openphone / openphones_not_standart
    return calls_after_openphone, openphones_not_standart, cv_from_openphones_to_calls

### We collect data on cr from all sessions to open_phone and from displaying open_phone to an actual call.

In [3]:
def get_cr_to_openphone(city, month):
    
    if city is None:
        print('CRs to openphones, All cities')
        city_clickhouse = ''
        
    else:
        print('CRs to openphones, ' + city)
        city_clickhouse = f"and seotarget_city in '{city}'"
        
#     cr_to_openphone = clickhouse(f"""
#         select 
#         uniqIf(session_id, ev_type = 'pageview') as traffic,
#         uniqIf(session_id, ev_type = 'event' and ev_action = 'display' and ev_category like 'phone_o%') as openphone_display,
#         uniqIf(session_id, ev_type = 'event' and ev_action = 'open' and ev_category like 'phone_o%') as openphone,
#         openphone / traffic as cr_session_openphone,
#         openphone / openphone_display as cr_display_openphone
#         from zoon.stat
#         where date_trunc('month', event_date) = '{month}'
#         and user_class = ''
#         and ua_type = 'desktop'
#         {city_clickhouse}
#         and page_url like '%zoon.%'
#         and ev_type in ('pageview', 'event')
#     """)

    cr_to_openphone = clickhouse(f"""
        select 
            uniq(session_id) as traffic, 
            uniqIf(session_id, ev_action = 'display' and event_time > catalog_time) as openphone_display, 
            uniqIf(session_id, ev_action = 'open' and event_time > catalog_time) as openphone,
            openphone / traffic as cr_session_openphone,
            openphone / openphone_display as cr_display_openphone
        from
        (
        select session_id, event_time catalog_time
        from zoon.stat
        where date_trunc('month', event_date) = '{month}'
            and page_url like '%zoon.%'
            and seotarget_type = 'catalog'
            and ev_type = 'pageview'
            and user_class = ''
            and ua_type = 'desktop'
            {city_clickhouse}
            ) as catalogs
        left join
        (select session_id, ev_action, event_time
        from zoon.stat
        where 
            date_trunc('month', event_date) = '{month}'
            and ev_type = 'event'
            and user_class = ''
            and page_url like '%zoon.%'
            {city_clickhouse}
            and ev_category like 'phone_o%'
            and ev_action in ('display', 'open')
            and session_id in         
                (select session_id
                from zoon.stat
                where date_trunc('month', event_date) = '{month}'
                    and page_url like '%zoon.%'
                    and seotarget_type = 'catalog'
                    and ev_type = 'pageview'
                    and user_class = ''
                    and ua_type = 'desktop'
                    {city_clickhouse}
                    ) as catalogs_2
            ) as events
        using(session_id)
    """)
    
    traffic = cr_to_openphone['traffic'][0]

    openphone = cr_to_openphone['openphone'][0]
    cr_session_openphone = cr_to_openphone['cr_session_openphone'][0]

    openphone_display = cr_to_openphone['openphone_display'][0]
    cr_display_openphone = cr_to_openphone['cr_display_openphone'][0]
    
    return traffic, openphone, cr_session_openphone, openphone_display, cr_display_openphone


### We gather data on cr from the catalog to an organization's profile card

In [4]:
def get_cr_from_catalog(city, month):
    if city is None:
        print('CR from catalog to org, All cities')
        city_clickhouse = ''
    else:
        print('CR from catalog to org, ' + city)
        city_clickhouse = f"and seotarget_city = '{city}'"
        
    cr_from_catalog = clickhouse(f"""
    select 
        uniqIf(session_id, ev_type = 'pageview') as catalog_traffic,
        uniqIf(session_id, ev_type = 'event' and ev_action = 'click' and ev_category = 'organization' and ev_sourceType = 'mini_card' and object_type = 'organization') as mini_card_click,
        mini_card_click / catalog_traffic as cr_catalog_mini_card
    from zoon.stat
    where date_trunc('month', event_date) = '{month}'
        and user_class = ''
        and seotarget_type = 'catalog'
        and ua_type = 'desktop'
        {city_clickhouse}
        and ev_type in ('pageview', 'event')
        and page_url like '%zoon.%'
    """)

    
    catalog_traffic = cr_from_catalog['catalog_traffic'][0]
    mini_card_click = cr_from_catalog['mini_card_click'][0]
    cr_catalog_mini_card = cr_from_catalog['cr_catalog_mini_card'][0]
    
    return catalog_traffic, mini_card_click, cr_catalog_mini_card

### Writing functions to calculate the required sample size and the necessary number of days for the experiment to determine absolute and relative effects.

In [5]:
# Function to Calculate the Time Required for a Test When Specifying an Absolute Effect.
def need_group_size_absolute(month_size, actual_conv_rate, min_effect_size, alpha, power):
    
    effect_size = proportion_effectsize(actual_conv_rate, actual_conv_rate + min_effect_size)
    # calculate the sample size for each group
    sample_size = zt_ind_solve_power(effect_size=effect_size, alpha=alpha, power=power, alternative='two-sided')
    # round the sample size to the nearest whole number
    rounded_sample_size = round(sample_size)
    days = (rounded_sample_size * 2) / (month_size / 30)
    
    print(f'To conduct an experiment with a significance level of {alpha}, power of {power}, and an absolute effect size of {min_effect_size:.1%}, a minimum group size of {rounded_sample_size} is required, and this will take {days:.0f} days.

    
    
    # функция, для подсчета времени, необходимого на тест при указании относительного эффекта
def need_group_size(month_size, actual_conv_rate, min_effect_size, alpha, power):
    control_conv_rate = actual_conv_rate
    treatment_conv_rate = control_conv_rate * (1 + min_effect_size)
    mde = (treatment_conv_rate - control_conv_rate) / control_conv_rate
    effect_size = proportion_effectsize(treatment_conv_rate, control_conv_rate)
    sample_size = zt_ind_solve_power(effect_size=effect_size, nobs1=None, alpha=alpha, power=power, ratio=1.0, alternative='two-sided')
    rounded_sample_size = round(sample_size)
    days = (rounded_sample_size * 2) / (month_size / 30)
    
    print(f"""
Alpha: {alpha}
Power: {power} 
Rel. MDE: {min_effect_size:.1%} 
Group Size: {rounded_sample_size}
Estimated test duration: {days:.0f}
If no changes detected we may loose: {int(round(month_size * actual_conv_rate - (actual_conv_rate - min_effect_size * actual_conv_rate) * month_size, 0))}
""")

### Задаем город и месяц и достаем данные из баз

In [6]:
city= 'spb'
month='2022-11-01'
calls_after_openphone, openphones_not_standart, cv_from_openphones_to_calls = get_openphones_and_calls(city,month)
traffic, openphone, cr_session_openphone, openphone_display, cr_display_openphone = get_cr_to_openphone(city, month)
catalog_traffic, mini_card_click, cr_catalog_mini_card = get_cr_from_catalog(city, month)

CR from openphones to calls, spb
CRs to openphones, spb
CR from catalog to org, spb


- alpha - вероятность обнаружить различия там, где их нет (на самом деле нулевая гипотеза верна, но мы ее отвергаем)- ошибка 1 рода
- beta - вероятность не обнаружить различия там, где они присутствуют (гипотеза не верна, но мы ее не отвергли)- ошибка 2 рода

В нашем тесте, нам важно, сделать тест более мощным, чтоб уменьшить вероятность того, что мы не обноружим значимых различий, там где они есть, то есть сократить вероятность ошибки второго рода.

### Применяем функцию подсчета необходимых дней эксперемента для определения относительного эффекта изменения конверсии к нашим данным

In [7]:
# print(f'Опенфонов в месяц по СПб для орг с экпаерами: {openphones_not_standart}')
# print(f'Звонков в месяц после опенфонов СПб: {calls_after_openphone }')
print(f'Конверсия из опенфона в звонок по {city}: {cv_from_openphones_to_calls:.1%}')
need_group_size(openphones_not_standart, cv_from_openphones_to_calls, min_effect_size=0.12, alpha=0.05, power=0.80)
print('----------------------------------------------------------')
print()
# print(f'Количество сессий в месяц с десктопа по СПб : {traffic}')
# print(f'Опенфонов в месяц с десктопа по СПб: {openphone}')
print(f'Конверсия из сессии в опенфон по {city}: {cr_session_openphone:.1%}')

need_group_size(traffic, cr_session_openphone, min_effect_size=0.04, alpha=0.01, power=0.80)
print('----------------------------------------------------------')
print()
# print(f'Количество показов опенфонов с десктопа в месяц по СПб : {openphone_display}')
# print(f'Опенфонов в месяц с десктопа по СПб: {openphone}')
print(f'Конверсия из показа в опенфон по {city}: {cr_display_openphone:.1%}')
print()
need_group_size(openphone_display, cr_display_openphone, min_effect_size=0.035, alpha=0.01, power=0.80)
print('----------------------------------------------------------')
print()
# print(f'Количество просмотров каталога с десктопа в месяц по СПб : {catalog_traffic}')
# print(f'Количество кликов по мини-карточке в месяц с десктопа по СПб: {mini_card_click}')
print(f'Конверсия из каталога в карточку орги по {city}: {cr_catalog_mini_card:.1%}')
print()
need_group_size(catalog_traffic, cr_catalog_mini_card, min_effect_size=0.02, alpha=0.01, power=0.80)

Конверсия из опенфона в звонок по spb: 17.5%

Alpha: 0.05
Power: 0.8 
Rel. MDE: 12.0% 
Group Size: 5359
Estimated test duration: 67
If no changes detected we may loose: 101

----------------------------------------------------------

Конверсия из сессии в опенфон по spb: 8.7%

Alpha: 0.01
Power: 0.8 
Rel. MDE: 4.0% 
Group Size: 156883
Estimated test duration: 64
If no changes detected we may loose: 505

----------------------------------------------------------

Конверсия из показа в опенфон по spb: 22.5%


Alpha: 0.01
Power: 0.8 
Rel. MDE: 3.5% 
Group Size: 66375
Estimated test duration: 71
If no changes detected we may loose: 442

----------------------------------------------------------

Конверсия из каталога в карточку орги по spb: 25.6%


Alpha: 0.01
Power: 0.8 
Rel. MDE: 2.0% 
Group Size: 171120
Estimated test duration: 70
If no changes detected we may loose: 746



In [8]:
# выводы ниже сейчас не актуальны

## Сравнение контрольной и тестовой групп

### Проверка гипотезы о равенстве конверсий из опенфона монетизируемой организации в звонок у контрольной и тестовой групп

In [9]:
start_day = '2023-03-07'

In [10]:
# Определение принадлежности сессии к одной из двух групп
query = f"""
select 
    session_id,
    anon_user_id,
    visitParamExtractString(ev_extra, 'ab_test_desktop_catalog_openphone_button_march_2023') as var,
    argMin(seotarget_type, event_time) as first_seotarget_type
from zoon.stat
where event_date >= '{start_day}'
    and user_class = ''
    and ua_type = 'desktop'
    and page_url like '%zoon.ru%'
    and ev_type in ('pageview', 'event')
    and visitParamExtractString(ev_extra, 'ab_test_desktop_catalog_openphone_button_march_2023') != ''
group by session_id, var, anon_user_id
having first_seotarget_type = 'catalog'
"""

ch_data = clickhouse(query)
display(ch_data.shape, ch_data.head())

(429527, 4)

,session_id,anon_user_id,var,first_seotarget_type
0,7526ce6e644d792ccb9af045417548,20230402204803QlZK.46bf,control,catalog
1,7e1ae0056438b30fa36bc522490659,20230414045735CyR0.f739,variation,catalog
2,099a94026433b5478334b083336475,20230410100543bWQP.b477,variation,catalog
3,68dca9b164522b8871945038708158,202305031238168oTd.cdfa,variation,catalog
4,5ade9638645da5584148d008673133,20230512053256MKWh.9912,control,catalog


In [11]:
openphones_all = clickhouse(f"""
        select *
        from
        (
        select session_id, event_time catalog_time
        from zoon.stat
        where event_date >= '{start_day}'
            and page_url like '%zoon.%'
            and seotarget_type = 'catalog'
            and ev_type = 'pageview'
            and user_class = '' -- добавил позже
            and ua_type = 'desktop' -- добавил позже
            and seotarget_city = 'spb'
            ) as catalogs
        inner join
        (select session_id, event_time open_phone_time, event_date, anon_user_id, object_id as org_id
        from zoon.stat
        where event_date >= '{start_day}'
            and page_url like '%zoon.%'
            and ev_type = 'event'
            and seotarget_city = 'spb'
            and ua_type = 'desktop'
            and ev_action = 'open'
            and ev_category like 'phone_o%'
            and ev_label not like '%standard%'
            and session_id in (
                select distinct session_id
                from zoon.stat
                where event_date >= '{start_day}'
                    and page_url like '%zoon.%'
                    and ev_type = 'pageview'
                    and seotarget_city = 'spb'
            and ua_type = 'desktop'
            and user_class = ''  -- добавил позже
            and seotarget_type = 'catalog') as catalogs_2
             ) as openphones
        using(session_id)
        where catalog_time < open_phone_time
""")

In [12]:
calls_all = vertica(f"""
    with cits as (
        select distinct _id 
        from zoon.organization 
        where 1=1
            and city = 'spb'
    )

    select  
        _id, 
        ts, 
        date_trunc('day', ts) as event_date,
        duration,
        profit_type,
        "approx_user.anon_user_id" as anon_user_id,
        org_id
    from zoon.stat_mangotelecom
    where date_trunc('day', ts) >= '{start_day}'
        and org_id in (select * from cits)
        and "real_type.object_id" is null 
        -- and source is null
        and org_phone != phone_from 
        and "approx_user.anon_user_id" is not null
        and "approx_user.anon_user_id" != ''
    """)
 


In [13]:
openphones_all = openphones_all.drop_duplicates(['session_id', 'open_phone_time', 'anon_user_id', 'org_id'])
openphones_all = openphones_all.merge(ch_data[['session_id', 'anon_user_id', 'var']], on=['session_id', 'anon_user_id'], how='inner')

openphones_control = openphones_all[openphones_all['var']=='control']
openphones_variation = openphones_all[openphones_all['var']=='variation']

df_all = openphones_all.merge(calls_all, how='inner', on=['anon_user_id', 'org_id', 'event_date']).drop_duplicates()


openphones_and_calls_control = df_all[df_all['var']=='control']
openphones_and_calls_variation = df_all[df_all['var']=='variation']

calls_after_openphone_all = df_all['_id'].nunique()
calls_after_openphone_control = openphones_and_calls_control['_id'].nunique()
calls_after_openphone_variation = openphones_and_calls_variation['_id'].nunique()


openphones_not_standart_all = len(openphones_all)
openphones_not_standart_control = len(openphones_control)
openphones_not_standart_variation = len(openphones_variation)

cv_from_openphones_to_calls_all = calls_after_openphone_all / openphones_not_standart_all
cv_from_openphones_to_calls_control = calls_after_openphone_control / openphones_not_standart_control
cv_from_openphones_to_calls_variation = calls_after_openphone_variation / openphones_not_standart_variation

In [14]:
openphones_all

,session_id,catalog_time,open_phone_time,event_date,anon_user_id,org_id,var
0,574377ee643e50066f2ba108077569,2023-04-18 11:08:46,2023-04-18 11:09:11,2023-04-18,20230129181534NQlQ.ab4c,58b59e1e0ab1eb62048b456e,variation
1,574377ee643e50066f2ba108077569,2023-04-18 11:08:46,2023-04-18 11:11:31,2023-04-18,20230129181534NQlQ.ab4c,5d5af83110de9a13911ccfe9,variation
2,5123f25964274c3c59aee259062697,2023-04-01 00:10:22,2023-04-01 00:10:43,2023-04-01,20230325233924QHDX.6841,598190674f135060764b81aa,control
3,c1b3cdd0643e50bbec8be404113578,2023-04-18 11:11:43,2023-04-18 11:19:44,2023-04-18,20230418111140JWu3.0847,5d5af8d610de9a13911cd38a,variation
4,235ae66f643e50bd011fe017318910,2023-04-18 11:11:51,2023-04-18 11:15:41,2023-04-18,20221203133509eoHU.4134,5154fa8ea0f302b66400004e,variation
...,...,...,...,...,...,...,...
13579,3d44fd39646f3cf53e140561159252,2023-05-25 13:48:22,2023-05-25 13:48:53,2023-05-25,20230525134821bxwO.b53b,61f583e4c97d9d6b746f668e,control
13580,2794c32a646f3b21105f0217536092,2023-05-25 13:48:41,2023-05-25 13:50:51,2023-05-25,20230316101100HUB0.aeef,5a087849a24fd923f30c7776,control
13581,2794c32a646f3b21105f0217536092,2023-05-25 13:48:41,2023-05-25 13:51:46,2023-05-25,20230316101100HUB0.aeef,5a087849a24fd923f30c7776,control
13582,94fe4996646f3eaa5ab70162297963,2023-05-25 13:55:40,2023-05-25 13:58:04,2023-05-25,202207080950445SkM.3d41,5167bed5a0f302a573000007,variation


In [15]:
print(f"""За время теста в СПб пользователи контрольной группы смотрели телефоны премиальных организаций {openphones_not_standart_control} раз, после чего совершили {calls_after_openphone_control} звонков
Конверсия в звонок в контрольной группе составила {cv_from_openphones_to_calls_control:.1%}""")
print()
print(f"""За время теста в СПб пользователи тестовой группы смотрели телефоны премиальных организаций {openphones_not_standart_variation} раз, после чего совершили {calls_after_openphone_variation} звонков
Конверсия в звонок в тестовой группе составила {cv_from_openphones_to_calls_variation:.1%}""")
print()

print('Проверим стат значимость различий конверсий')

alpha = 0.01
prob = 1 - alpha

stat, pvalue = proportions_ztest([calls_after_openphone_control, calls_after_openphone_variation], [openphones_not_standart_control, openphones_not_standart_variation])

print('P-value равен: {:.5f}'.format(pvalue))

if pvalue < alpha:
    print(f'Отклоняем нулевую гипотезу о равенстве конверсий в группах, c вероятностью более {prob:.0%} конверсии различаются')
else:
    print('Недостаточно оснований отклонить нулевую гипотезу о равенстве конверсий')

За время теста в СПб пользователи контрольной группы смотрели телефоны премиальных организаций 5818 раз, после чего совершили 1205 звонков
Конверсия в звонок в контрольной группе составила 20.7%

За время теста в СПб пользователи тестовой группы смотрели телефоны премиальных организаций 7766 раз, после чего совершили 1345 звонков
Конверсия в звонок в тестовой группе составила 17.3%

Проверим стат значимость различий конверсий
P-value равен: 0.00000
Отклоняем нулевую гипотезу о равенстве конверсий в группах, c вероятностью более 99% конверсии различаются


#### Вывод:
Конверсия из опенфона монетизируемой организации в звонок в эту организацию в тестовой группе статистически значимо ниже чем в контрольной группе

### Проверка гипотез о равенстве конверсий из сессии в показ опенфона а также из показа опенфона в опенфон для тестовой и контрольной групп

In [16]:
# считаем конверсии из сессий в показа опенфона и в опенфон из показа опенфона для контрольной группы
# сессии контрольной группы выбираются в последнем подзапросе
cr_to_openphone_control = clickhouse(f"""
        select 
    uniq(session_id) as traffic, 
    uniqIf(session_id, ev_action = 'display' and event_time > catalog_time) as openphone_display, 
    uniqIf(session_id, ev_action = 'open' and event_time > catalog_time) as openphone,
    openphone / traffic as cr_session_openphone,
    openphone / openphone_display as cr_display_openphone
from
(
select session_id, event_time catalog_time
from zoon.stat
where event_date >= '{start_day}'
    and page_url like '%zoon.%'
    and seotarget_type = 'catalog'
    and ev_type = 'pageview'
    and user_class = ''
    and ua_type = 'desktop'
    and seotarget_city = 'spb'
    and session_id in (
        select distinct session_id
        from 
        (select 
            session_id,
            anon_user_id,
            visitParamExtractString(ev_extra, 'ab_test_desktop_catalog_openphone_button_march_2023') as var,
            argMin(seotarget_type, event_time) as first_seotarget_type
        from zoon.stat
        where event_date >= '{start_day}'
            and user_class = ''
            and ua_type = 'desktop'
            and page_url like '%zoon.ru%'
            and ev_type in ('pageview', 'event')
            and visitParamExtractString(ev_extra, 'ab_test_desktop_catalog_openphone_button_march_2023') in ('control')
        group by session_id, var, anon_user_id
        having first_seotarget_type = 'catalog') dd
    )
    ) as catalogs
left join
(select session_id, ev_action, event_time
from zoon.stat
where 
    event_date >= '2023-03-07'
    and ev_type = 'event'
    and user_class = ''
    and page_url like '%zoon.%'
    and seotarget_city = 'spb'
    and ev_category like 'phone_o%'
    and ev_action in ('display', 'open')
    and session_id in         
        (select session_id
        from zoon.stat
        where event_date >= '{start_day}'
            and page_url like '%zoon.%'
            and seotarget_type = 'catalog'
            and ev_type = 'pageview'
            and user_class = ''
            and ua_type = 'desktop'
            and seotarget_city = 'spb'
            ) as catalogs_2
    ) as events
using(session_id)
    """)
    
traffic_control = cr_to_openphone_control['traffic'][0]

openphone_control = cr_to_openphone_control['openphone'][0]
cr_session_openphone_control = cr_to_openphone_control['cr_session_openphone'][0]

openphone_display_control = cr_to_openphone_control['openphone_display'][0]
cr_display_openphone_control = cr_to_openphone_control['cr_display_openphone'][0]

In [17]:
# считаем конверсии из сессий в показа опенфона и в опенфон из показа опенфона для тестовой группы
cr_to_openphone_variation = clickhouse(f"""
select 
    uniq(session_id) as traffic, 
    uniqIf(session_id, ev_action = 'display' and event_time > catalog_time) as openphone_display, 
    uniqIf(session_id, ev_action = 'open' and event_time > catalog_time) as openphone,
    openphone / traffic as cr_session_openphone,
    openphone / openphone_display as cr_display_openphone
from
(
select session_id, event_time catalog_time
from zoon.stat
where event_date >= '{start_day}'
    and page_url like '%zoon.%'
    and seotarget_type = 'catalog'
    and ev_type = 'pageview'
    and user_class = ''
    and ua_type = 'desktop'
    and seotarget_city = 'spb'
    and session_id in (
        select distinct session_id
        from 
        (select 
            session_id,
            anon_user_id,
            visitParamExtractString(ev_extra, 'ab_test_desktop_catalog_openphone_button_march_2023') as var,
            argMin(seotarget_type, event_time) as first_seotarget_type
        from zoon.stat
        where event_date >= '{start_day}'
            and user_class = ''
            and ua_type = 'desktop'
            and page_url like '%zoon.ru%'
            and ev_type in ('pageview', 'event')
            and visitParamExtractString(ev_extra, 'ab_test_desktop_catalog_openphone_button_march_2023') in ('variation')
        group by session_id, var, anon_user_id
        having first_seotarget_type = 'catalog') dd
    )
    ) as catalogs
left join
(select session_id, ev_action, event_time
from zoon.stat
where 
    event_date >= '{start_day}'
    and ev_type = 'event'
    and user_class = ''
    and page_url like '%zoon.%'
    and seotarget_city = 'spb'
    and ev_category like 'phone_o%'
    and ev_action in ('display', 'open')
    and session_id in         
        (select session_id
        from zoon.stat
        where event_date >= '{start_day}'
            and page_url like '%zoon.%'
            and seotarget_type = 'catalog'
            and ev_type = 'pageview'
            and user_class = ''
            and ua_type = 'desktop'
            and seotarget_city = 'spb'
            ) as catalogs_2
    ) as events
using(session_id)
    """)
    
traffic_variation = cr_to_openphone_variation['traffic'][0]

openphone_variation = cr_to_openphone_variation['openphone'][0]
cr_session_openphone_variation = cr_to_openphone_variation['cr_session_openphone'][0]

openphone_display_variation = cr_to_openphone_variation['openphone_display'][0]
cr_display_openphone_variation = cr_to_openphone_variation['cr_display_openphone'][0]


Сравним конверсию из кол-ва сессий в опенфоны

In [18]:
print(f"""Кол-во сессий за время теста в контрольной группе {traffic_control}, кол-во опенфонов в контрольной группе {openphone_control}
Конверсия из сессии в опенфон в контрольной группе {cr_session_openphone_control:.1%}""")
print()
print(f"""Кол-во сессий за время теста в тестовой группе {traffic_variation}, кол-во опенфонов в контрольной группе {openphone_variation}
Конверсия из сессии в опенфон в тестовой группе {cr_session_openphone_variation:.1%}""")
print()
alpha = 0.01
prob = 1 - alpha

stat, pvalue = proportions_ztest([openphone_display_control, openphone_display_variation], [traffic_control, traffic_variation])

print('P-value равен: {:.5f}'.format(pvalue))

if pvalue < alpha:
    print(f'Отклоняем нулевую гипотезу о равенстве конверсий в группах, c вероятностью более {prob:.0%} конверсии различаются')
else:
    print('Недостаточно оснований отклонить нулевую гипотезу о равенстве конверсий')

Кол-во сессий за время теста в контрольной группе 200993, кол-во опенфонов в контрольной группе 15764
Конверсия из сессии в опенфон в контрольной группе 7.8%

Кол-во сессий за время теста в тестовой группе 203035, кол-во опенфонов в контрольной группе 17817
Конверсия из сессии в опенфон в тестовой группе 8.8%

P-value равен: 0.00000
Отклоняем нулевую гипотезу о равенстве конверсий в группах, c вероятностью более 99% конверсии различаются


In [19]:
print(f"""Кол-во показов опенфонов за время теста в контрольной группе {openphone_display_control}, кол-во опенфонов в контрольной группе {openphone_control}
Конверсия из показа в опенфон в контрольной группе {cr_display_openphone_control:.1%}""")
print()
print(f"""Кол-во показов опенфонов за время теста в тестовой группе {openphone_display_variation}, кол-во опенфонов в тестовой группе {openphone_variation}
Конверсия из показа в опенфон в тестовой группе {cr_display_openphone_variation:.1%}""")

Кол-во показов опенфонов за время теста в контрольной группе 69091, кол-во опенфонов в контрольной группе 15764
Конверсия из показа в опенфон в контрольной группе 22.8%

Кол-во показов опенфонов за время теста в тестовой группе 158760, кол-во опенфонов в тестовой группе 17817
Конверсия из показа в опенфон в тестовой группе 11.2%


Статистический тест можно не применять, значения явно отличаются на стат значимую величину

#### Вывод:
Конверсия из сессии в опенфон в тестовой группе выросла на статистически значимую велечину,
а конверсия из показа опенфона в опенфон в тестовой группе сильно просела относительно контрольной.

### Проверка гипотезы о равенстве конверсий из всех сессий в звонок в монетезируемые организации

In [20]:
alpha = 0.05
prob = 1 - alpha

cr_from_session_to_not_standart_call_control = calls_after_openphone_control / traffic_control 
cr_from_session_to_not_standart_call_variation = calls_after_openphone_variation / traffic_variation

ci_control = proportion_confint(calls_after_openphone_control, traffic_control, alpha=alpha, method='normal')
ci_variation = proportion_confint(calls_after_openphone_variation, traffic_variation, alpha=alpha, method='normal')

print(f"""Кол-во сессий за время теста в контрольной группе {traffic_control}, кол-во звонков в монитезируемые организации {calls_after_openphone_control}
Конверсия из сессии в звонок в монетезируемые организации в контрольной группе {cr_from_session_to_not_standart_call_control:.2%} и с вероятностью {prob:.0%} находится в интервале {ci_control[0]:0.4%} - {ci_control[1]:0.4%}""")
print()
print(f"""Кол-во сессий за время теста в тестовой группе {traffic_variation}, кол-во звонков в монитезируемые организации {calls_after_openphone_variation}
Конверсия из сессии в в звонок в монетезируемые организации в тестовой группе {cr_from_session_to_not_standart_call_variation:.2%} и с вероятностью {prob:.0%} находится в интервале {ci_variation[0]:0.4%} - {ci_variation[1]:0.4%}""")
print()


stat, pvalue = proportions_ztest([calls_after_openphone_control, calls_after_openphone_variation], [traffic_control, traffic_variation])


print('P-value равен: {:.5f}'.format(pvalue))

if pvalue < alpha:
    print(f'Отклоняем нулевую гипотезу о равенстве конверсий в группах, c вероятностью более {prob:.0%} конверсии различаются')
else:
    print('Недостаточно оснований отклонить нулевую гипотезу о равенстве конверсий')

Кол-во сессий за время теста в контрольной группе 200993, кол-во звонков в монитезируемые организации 1205
Конверсия из сессии в звонок в монетезируемые организации в контрольной группе 0.60% и с вероятностью 95% находится в интервале 0.5658% - 0.6333%

Кол-во сессий за время теста в тестовой группе 203035, кол-во звонков в монитезируемые организации 1345
Конверсия из сессии в в звонок в монетезируемые организации в тестовой группе 0.66% и с вероятностью 95% находится в интервале 0.6272% - 0.6977%

P-value равен: 0.01156
Отклоняем нулевую гипотезу о равенстве конверсий в группах, c вероятностью более 95% конверсии различаются


#### Вывод:
Со статистически значимой вероятностью можем говорить о том, что кол-во звонков в монетизируемые организации в тестовой группе выше, чем в контрольной при том же трафике

### Проверка гипотезы о равенстве конверсий из каталога в карточку организации в тестовой и контрольной группах

In [21]:
cr_from_catalog_control = clickhouse(f"""
    select 
        uniqIf(session_id, ev_type = 'pageview') as catalog_traffic,
        uniqIf(session_id, ev_type = 'event' and ev_action = 'click' and ev_category = 'organization' and ev_sourceType = 'mini_card' and object_type = 'organization') as mini_card_click,
        mini_card_click / catalog_traffic as cr_catalog_mini_card
    from zoon.stat
    where event_date >= '{start_day}'
        and user_class = ''
        and seotarget_type = 'catalog'
        and ua_type = 'desktop'
        and seotarget_city = 'spb'
        and ev_type in ('pageview', 'event')
        and page_url like '%zoon.%'
        and session_id in (select distinct session_id
            from
             (select
                session_id,
                argMin(seotarget_type, event_time) as first_seo
            from zoon.stat 
            where event_date >= '{start_day}'
                and visitParamExtractString(ev_extra, 'ab_test_desktop_catalog_openphone_button_march_2023') = 'control'
                and seotarget_city = 'spb'
                and ua_type = 'desktop'
            group by session_id) as first_seo_query
            where first_seo = 'catalog')
    """)

    
catalog_traffic_control = cr_from_catalog_control['catalog_traffic'][0]
mini_card_click_control = cr_from_catalog_control['mini_card_click'][0]
cr_catalog_mini_card_control = cr_from_catalog_control['cr_catalog_mini_card'][0]

In [22]:
cr_from_catalog_variation = clickhouse(f"""
    select 
        uniqIf(session_id, ev_type = 'pageview') as catalog_traffic,
        uniqIf(session_id, ev_type = 'event' and ev_action = 'click' and ev_category = 'organization' and ev_sourceType = 'mini_card' and object_type = 'organization') as mini_card_click,
        mini_card_click / catalog_traffic as cr_catalog_mini_card
    from zoon.stat
    where event_date >= '{start_day}'
        and user_class = ''
        and seotarget_type = 'catalog'
        and ua_type = 'desktop'
        and seotarget_city = 'spb'
        and ev_type in ('pageview', 'event')
        and page_url like '%zoon.%'
        and session_id in (select distinct session_id
            from
             (select
                session_id,
                argMin(seotarget_type, event_time) as first_seo
            from zoon.stat 
            where event_date >= '{start_day}'
                and visitParamExtractString(ev_extra, 'ab_test_desktop_catalog_openphone_button_march_2023') = 'variation'
                and seotarget_city = 'spb'
                and ua_type = 'desktop'
            group by session_id) as first_seo_query
            where first_seo = 'catalog')
    """)

    
catalog_traffic_variation = cr_from_catalog_variation['catalog_traffic'][0]
mini_card_click_variation = cr_from_catalog_variation['mini_card_click'][0]
cr_catalog_mini_card_variation = cr_from_catalog_variation['cr_catalog_mini_card'][0]

In [23]:
print(f"""Кол-во посещений каталога за время теста в контрольной группе {catalog_traffic_control}, кол-во переходов на карточку организации из каталога в контрольной группе {mini_card_click_control}
Конверсия из каталога в карточку организации в контрольной группе {cr_catalog_mini_card_control:.1%}""")
print()
print(f"""Кол-во посещений каталога за время теста в тестовой группе {catalog_traffic_variation}, кол-во переходов на карточку организации из каталога в тестовой группе {mini_card_click_variation}
Конверсия из каталога в карточку организации в тестовой группе {cr_catalog_mini_card_variation:.1%}""")
print()

alpha = 0.01
prob = 1 - alpha

stat, pvalue = proportions_ztest([mini_card_click_control, mini_card_click_variation], [catalog_traffic_control, catalog_traffic_variation])

print('P-value равен: {:.5f}'.format(pvalue))

if pvalue < alpha:
    print(f'Отклоняем нулевую гипотезу о равенстве конверсий в группах, c вероятностью более {prob:.0%} конверсии различаются')
else:
    print('Недостаточно оснований отклонить нулевую гипотезу о равенстве конверсий')

Кол-во посещений каталога за время теста в контрольной группе 201081, кол-во переходов на карточку организации из каталога в контрольной группе 42793
Конверсия из каталога в карточку организации в контрольной группе 21.3%

Кол-во посещений каталога за время теста в тестовой группе 203147, кол-во переходов на карточку организации из каталога в тестовой группе 39391
Конверсия из каталога в карточку организации в тестовой группе 19.4%

P-value равен: 0.00000
Отклоняем нулевую гипотезу о равенстве конверсий в группах, c вероятностью более 99% конверсии различаются


#### Вывод:
Конверсия перехода из каталога в карточку организации в тестовой группе статистически значимо ниже, чем в контрольной.

### Проверка гипотезы о равенстве конверсии из сессии в опенфон на карточке организации (предполагаем возможную канибализацию этого трафика)

In [24]:
cr_to_openphone_in_service_control = clickhouse(f"""
select 
    uniq(session_id) as traffic, 
    uniqIf(session_id, ev_action = 'display' and event_time > catalog_time) as openphone_display, 
    uniqIf(session_id, ev_action = 'open' and event_time > catalog_time) as openphone,
    openphone / traffic as cr_session_openphone,
    openphone / openphone_display as cr_display_openphone
from
(
select session_id, event_time catalog_time
from zoon.stat
where event_date >= '{start_day}'
    and page_url like '%zoon.%'
    and seotarget_type = 'catalog'
    and ev_type = 'pageview'
    and user_class = ''
    and ua_type = 'desktop'
    and seotarget_city = 'spb'
    and session_id in (
        select distinct session_id
        from 
        (select 
            session_id,
            anon_user_id,
            visitParamExtractString(ev_extra, 'ab_test_desktop_catalog_openphone_button_march_2023') as var,
            argMin(seotarget_type, event_time) as first_seotarget_type
        from zoon.stat
        where event_date >= '{start_day}'
            and user_class = ''
            and ua_type = 'desktop'
            and page_url like '%zoon.ru%'
            and ev_type in ('pageview', 'event')
            and visitParamExtractString(ev_extra, 'ab_test_desktop_catalog_openphone_button_march_2023') in ('control')
        group by session_id, var, anon_user_id
        having first_seotarget_type = 'catalog') dd
    )
    ) as catalogs
left join
(select session_id, ev_action, event_time
from zoon.stat
where 
    event_date >= '{start_day}'
    and ev_type = 'event'
    and user_class = ''
    and page_url like '%zoon.%'
    and seotarget_city = 'spb'
    and ev_category like 'phone_o%'
    and ev_action in ('display', 'open')
    and seotarget_type = 'service'
    --and ev_sourceType = 'catalog'
    and session_id in         
        (select session_id
        from zoon.stat
        where event_date >= '{start_day}'
            and page_url like '%zoon.%'
            and seotarget_type = 'catalog'
            and ev_type = 'pageview'
            and user_class = ''
            and ua_type = 'desktop'
            and seotarget_city = 'spb'
            ) as catalogs_2
    ) as events
using(session_id)
    """)

In [25]:
cr_to_openphone_in_service_variation = clickhouse(f"""
select 
    uniq(session_id) as traffic, 
    uniqIf(session_id, ev_action = 'display' and event_time > catalog_time) as openphone_display, 
    uniqIf(session_id, ev_action = 'open' and event_time > catalog_time) as openphone,
    openphone / traffic as cr_session_openphone,
    openphone / openphone_display as cr_display_openphone
from
(
select session_id, event_time catalog_time
from zoon.stat
where event_date >= '{start_day}'
    and page_url like '%zoon.%'
    and seotarget_type = 'catalog'
    and ev_type = 'pageview'
    and user_class = ''
    and ua_type = 'desktop'
    and seotarget_city = 'spb'
    and session_id in (
        select distinct session_id
        from 
        (select 
            session_id,
            anon_user_id,
            visitParamExtractString(ev_extra, 'ab_test_desktop_catalog_openphone_button_march_2023') as var,
            argMin(seotarget_type, event_time) as first_seotarget_type
        from zoon.stat
        where event_date >= '{start_day}'
            and user_class = ''
            and ua_type = 'desktop'
            and page_url like '%zoon.ru%'
            and ev_type in ('pageview', 'event')
            and visitParamExtractString(ev_extra, 'ab_test_desktop_catalog_openphone_button_march_2023') in ('variation')
        group by session_id, var, anon_user_id
        having first_seotarget_type = 'catalog') dd
    )
    ) as catalogs
left join
(select session_id, ev_action, event_time
from zoon.stat
where 
    event_date >= '{start_day}'
    and ev_type = 'event'
    and user_class = ''
    and page_url like '%zoon.%'
    and seotarget_city = 'spb'
    and ev_category like 'phone_o%'
    and ev_action in ('display', 'open')
    and seotarget_type = 'service'
    --and ev_sourceType = 'catalog'
    and session_id in         
        (select session_id
        from zoon.stat
        where event_date >= '{start_day}'
            and page_url like '%zoon.%'
            and seotarget_type = 'catalog'
            and ev_type = 'pageview'
            and user_class = ''
            and ua_type = 'desktop'
            and seotarget_city = 'spb'
            ) as catalogs_2
    ) as events
using(session_id)
    """)

In [26]:
cr_to_openphone_in_service_control

,traffic,openphone_display,openphone,cr_session_openphone,cr_display_openphone
0,201012,59347,10622,0.052843,0.178981


In [27]:
cr_to_openphone_in_service_variation

,traffic,openphone_display,openphone,cr_session_openphone,cr_display_openphone
0,203064,55921,8582,0.042263,0.153466


Статистический тест можно не применять, значения явно отличаются на стат значимую величину

In [28]:
reduction_percentage_openphone_in_service = 1 - cr_to_openphone_in_service_variation['cr_session_openphone'][0] / cr_to_openphone_in_service_control['cr_session_openphone'][0]
reduction_percentage_display_openphone_in_service = 1 - cr_to_openphone_in_service_variation['cr_display_openphone'][0] / cr_to_openphone_in_service_control['cr_display_openphone'][0]

In [29]:
print(f'Падение конверсии из трафика в опенфон на карточке организации составляет {reduction_percentage_openphone_in_service:0.1%}')
print(f'Падение конверсии из показа опенфона в опенфон на карточке организации составляет {reduction_percentage_display_openphone_in_service:0.1%}')


Падение конверсии из трафика в опенфон на карточке организации составляет 20.0%
Падение конверсии из показа опенфона в опенфон на карточке организации составляет 14.3%


#### Вывод:
Как мы видим, в тестовой группе произошло заметное падение (около 20%) конверсии из сессии в опенфон в карточки организации и заметное (около 14%) падение конверсии из показа опенфона в карточке организации в открытие опенфона, что подтвердило наши догадки о возможном канибализации трафика.

## Общие выводы по A/B тесту

**В сессиях тестовой группы у нас просели некоторые метрики:**
- Конверсия из показа опенфона в опенфон
Контрольная группа: показов опенфонов - 67110, кликов по опенфонам - 15301, конверсия - 22.8%
Тестовая группа: показов опенфонов - 154482, кликов по опенфонам - 17293, конверсия - 11.2%
( У нас кол-во показов опенфонов выросло более чем в 2 раза, клики по опенфонам тоже выросли, но не так сильно, из-за этого конверсия из показа в открытие опенфона сократилась)

- Конверсия из опенфона монетизируемой организации в звонок в эту организацию
Контрольная группа: опенфонов - 5624, звонков - 1162, конверсия - 20.7%
Тестовая группа: опенфонов - 7536, звонков - 1296, конверсия - 17.2%
(Кол-во опенфонов у монетизируемых организаций выросло, кол-во звонков тоже выросло, но не так значительно, из-за чего конверсия сократилась)

- Конверсия из каталога в карточку организации
Контрольная группа: посещений каталога - 196067, переходов в карточку - 41562, конверсия - 21.2%
Тестовая группа: посещений каталога - 198136, переходов в карточку - 38345, конверсия - 19.4%
(Так как у пользователей появилась возможность посмотреть телефон в каталоге, немного сократилось кол-во переходов из каталога в карточку организации)

- Конверсия из сессии в клик по опенфонам в карточке организации
Контрольная группа: сессий - 196003, кликов по опенфону на карточке организации - 10326, конверсия - 5.3% 
Тестовая группа: группа: сессий - 198038, кликов по опенфону на карточке организации - 8329, конверсия - 4.2%
(Сократилось кол-во пользователей, переходящих в карточку организации, вслед за эти сократилось и количество опенфонов на карточке организации)

- Конверсия из показа опенфона в клик по опенфону на карточке организации:
Контрольная группа: показов опенфонов на карточке - 57660, опенфонов на карточке - 10326, конверсия - 17.9%
Тестовая группа: показов опенфонов на карточке - 54352, опенфонов на карточке - 8329, конверсия - 15.3%
(Уменьшение этой конверсии сложно объяснить, возможно, пользователи, которые изначально испытывают высокую потребность в номере телефона удовлетворяют ее теперь в каталоге, а на карточку организации доходят менее мотивированные на звонок пользователи)


**При этом в сессиях тестовой группы выросли некоторые метрики:**
- Конверсия из сессии в опенфон
Контрольная группа: сессий - 196010, опенфонов - 15301, конверия - 7.8%
Тестовва группа: сессий - 198043, опенфонов - 17293, конверия - 8.7%
(Общее кол-во опенфонов выросло, и это хорошо)

- Конверсия из сессии в звонк в монетезируемые организации
( Различия в конверсиях удалось подтвердить с вероятностью более 95%)
Контрольная группа: сессий - 196010, звонков 1162, конверсия - 0.59% (доверительный интервал: 0.56% - 0.63%)
Тестовая группа: сессий - 198043, звонков 1296, конверсия - 0.65% (доверительный интервал: 0.62% - 0.69%)
(Общее кол-во звонков в монетезируемые организации с большой долей вероятности выросло и почти наверняка не упало, что безусловно, является положительным результатом теста)


**Несмотря на падение некоторых второстепенных метрик, нам удалось установить рост двух ключевых мерик, а именно, общего кол-ва опенфонов и кол-ва звонков в монетезируемые организации. Таким образом предлагаю считать тест успешным и раскатить новую версию на все города.**